In [1]:
# Install the python library for delphi-epidata and covidcast
%pip install delphi-epidata
%pip install covidcast

     |████████████████████████████████| 12.3MB 5.7MB/s 
     |████████████████████████████████| 26.9MB 1.3MB/s 
     |████████████████████████████████| 972kB 40.5MB/s 
     |████████████████████████████████| 14.8MB 147kB/s 
     |████████████████████████████████| 6.5MB 44.4MB/s 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import covidcast
from delphi_epidata import Epidata
from datetime import date

# Helper function for getting different signals for model
# By default, after exploration, I realized that only using confirmed cases yielded the best results (based on RMSE)
def preprocess_covid_data(geotype):
  # # Early indicators
  # print("Full time work")
  # full_time_work = covidcast.signal("safegraph", "full_time_work_prop", geo_type=geotype)
  # print("Part time work")
  # part_time_work = covidcast.signal("safegraph", "part_time_work_prop", geo_type=geotype)
  # print("Covid Symptoms")
  # covid_symptoms = covidcast.signal("fb-survey", "smoothed_cli", geo_type=geotype)
  # print("Covid doctor visits")
  # covid_doctor_visits = covidcast.signal("doctor-visits", "smoothed_adj_cli", geo_type=geotype)
  # print("Covid indicator combination")
  # covid_indicator_combination = covidcast.signal("indicator-combination", "nmf_day_doc_fbc_fbs_ght", geo_type=geotype)

  # # Late indicators
  # print("Covid antigen test")
  # covid_antigen_test = covidcast.signal("quidel", "covid_ag_smoothed_pct_positive", geo_type=geotype)
  # print("Covid hospital admit")
  # covid_hospital_admit = covidcast.signal("hospital-admissions", "smoothed_adj_covid19_from_claims", geo_type=geotype)
  # print("Covid cases")
  # covid_cases = covidcast.signal("indicator-combination", "confirmed_7dav_incidence_num", geo_type=geotype)
  # print("Covid deaths")
  # covid_deaths = covidcast.signal("indicator-combination", "deaths_7dav_incidence_num", geo_type=geotype)

  # # Rename columns
  # full_time_work = full_time_work.rename(columns={"value": "full_time_work_prop"})
  # part_time_work = part_time_work.rename(columns={"value": "part_time_work_prop"})
  # covid_symptoms = covid_symptoms.rename(columns={"value": "smoothed_cli"})
  # covid_doctor_visits = covid_doctor_visits.rename(columns={"value": "smoothed_adj_cli"})
  # covid_indicator_combination = covid_indicator_combination.rename(columns={"value": "nmf_day_doc_fbc_fbs_ght"})
  # covid_antigen_test = covid_antigen_test.rename(columns={"value": "covid_ag_smoothed_pct_positive"})
  # covid_hospital_admit = covid_hospital_admit.rename(columns={"value": "smoothed_adj_covid19_from_claims"})
  # covid_cases = covid_cases.rename(columns={"value": "confirmed_7dav_incidence_num"})
  # covid_deaths = covid_deaths.rename(columns={"value": "deaths_7dav_incidence_num"})

  # # Merge columns
  # merged_data_df = pd.merge(full_time_work[["geo_value", "time_value", "full_time_work_prop"]], part_time_work[["geo_value", "time_value", "part_time_work_prop"]], on=["geo_value", "time_value"], how="inner")
  # merged_data_df = pd.merge(merged_data_df, covid_symptoms[["geo_value", "time_value", "smoothed_cli"]], on=["geo_value", "time_value"], how="inner")
  # merged_data_df = pd.merge(merged_data_df, covid_doctor_visits[["geo_value", "time_value", "smoothed_adj_cli"]], on=["geo_value", "time_value"], how="inner")
  # merged_data_df = pd.merge(merged_data_df, covid_indicator_combination[["geo_value", "time_value", "nmf_day_doc_fbc_fbs_ght"]], on=["geo_value", "time_value"], how="inner")
  # merged_data_df = pd.merge(merged_data_df, covid_antigen_test[["geo_value", "time_value", "covid_ag_smoothed_pct_positive"]], on=["geo_value", "time_value"], how="inner")
  # merged_data_df = pd.merge(merged_data_df, covid_hospital_admit[["geo_value", "time_value", "smoothed_adj_covid19_from_claims"]], on=["geo_value", "time_value"], how="inner")
  # merged_data_df = pd.merge(merged_data_df, covid_cases[["geo_value", "time_value", "confirmed_7dav_incidence_num"]], on=["geo_value", "time_value"], how="inner")
  # merged_data_df = pd.merge(merged_data_df, covid_deaths[["geo_value", "time_value", "deaths_7dav_incidence_num"]], on=["geo_value", "time_value"], how="inner")

  print("Covid cases")
  covid_cases = covidcast.signal("indicator-combination", "confirmed_7dav_incidence_num", geo_type=geotype)
  covid_cases = covid_cases.rename(columns={"value": "confirmed_cases"})

  return covid_cases

In [ ]:
%pip install joblib
import joblib
from sklearn.preprocessing import MinMaxScaler
columns = ["covid_symptoms", "confirmed_cases", "hospital_admits", "covid_test", "time_group"]
covid_features = columns[:-1] # [columns[7]] # columns[:-1]
merged_data_df = preprocess_covid_data("state")

# Scale the data based on the signals we have received
case_scaler = MinMaxScaler(feature_range=(0,1))
merged_data_df[["confirmed_cases"]] = case_scaler.fit_transform(merged_data_df[["confirmed_cases"]])
merged_data_df

Covid cases


,geo_value,signal,time_value,issue,lag,confirmed_cases,stderr,sample_size,geo_type,data_source
0,ak,confirmed_7dav_incidence_num,2020-02-20,2020-07-10,141,0.014130,None,None,state,indicator-combination
1,al,confirmed_7dav_incidence_num,2020-02-20,2020-07-10,141,0.014130,None,None,state,indicator-combination
2,ar,confirmed_7dav_incidence_num,2020-02-20,2020-07-10,141,0.014130,None,None,state,indicator-combination
3,az,confirmed_7dav_incidence_num,2020-02-20,2020-07-10,141,0.014130,None,None,state,indicator-combination
4,ca,confirmed_7dav_incidence_num,2020-02-20,2020-07-10,141,0.014137,None,None,state,indicator-combination
...,...,...,...,...,...,...,...,...,...,...
47,vt,confirmed_7dav_incidence_num,2020-12-06,2020-12-07,1,0.020459,None,None,state,indicator-combination
48,wa,confirmed_7dav_incidence_num,2020-12-06,2020-12-07,1,0.116129,None,None,state,indicator-combination
49,wi,confirmed_7dav_incidence_num,2020-12-06,2020-12-07,1,0.235550,None,None,state,indicator-combination
50,wv,confirmed_7dav_incidence_num,2020-12-06,2020-12-07,1,0.069470,None,None,state,indicator-combination


In [ ]:
import zipfile

# Helper function to put the information from COVIDCast to separate CSV files by state and then compress into a ZIP file
def write_covid_to_csv(merged_data_df):
  with zipfile.ZipFile('covid_states_day.zip', 'w') as csv_zip:
    for state in merged_data_df["geo_value"].unique():
      print(state)
      state_df = merged_data_df[merged_data_df['geo_value'].isin([state])]
      state_df = state_df.sort_values(by='time_value')
      state_timevals = state_df['time_value']
      day = pd.Timedelta('1d')
      breaks = state_timevals.diff() != day
      groups = breaks.cumsum()
      state_df['time_group'] = groups
      print(state_df)

      # Write output to csv in zip file 
      csv_zip.writestr("{}.csv".format(state), state_df.to_csv())
      
write_covid_to_csv(merged_data_df)
joblib.dump(case_scaler, 'case_scaler.bin', compress=True)
# joblib.dump(scaler, 'scaler.bin', compress=True)

ak
   geo_value                        signal  ...            data_source time_group
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
..       ...                           ...  ...                    ...        ...
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1
0         ak  confirmed_7dav_incidence_num  ...  indicator-combination          1

[291 rows x 

['scaler.bin']

In [ ]:
n_days = 3 # We use data from the past 1 week(s) to predict the next day
covid_features = ["confirmed_cases"] # [columns[7]] # columns[:-1]

# Read the individual CSVs and preprocess them as training input and labels
# Input: A set of n_days consecutive np arrays with the features listed in covid_features
# Label: The following day from the n_days consecutive days
# Output: Both the input vectors (dataX) and output labels (dataY)
def get_consecutive_date_features(state_covid_df):
  dataX, dataY = [], []
  for value in state_covid_df["time_group"].unique():
    state_covid_df_group = state_covid_df[state_covid_df["time_group"].isin([value])]
    for i in range(len(state_covid_df_group)-n_days-predict_days-1):
      arr = state_covid_df_group[covid_features].iloc[i:i+n_days].values.tolist()
      dataX.append(arr)
      dataY.append(state_covid_df_group[covid_features].iloc[i+n_days].loc['confirmed_cases'])

  return np.array(dataX), np.array(dataY)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import date
import zipfile

dataX, dataY = np.array([]), np.array([])
cases_only_columns = ["confirmed_7dav_incidence_num", "time_group"]

# Read the zip files and aggregate all the inputs and outputs
with zipfile.ZipFile("covid_states_day.zip", "r") as f:
    for name in f.namelist():
        if ".csv" in name:
          print("Fetching data from {}...".format(name))
          covid_df = pd.read_csv(f.open(name), low_memory=False, usecols=["confirmed_cases", "time_group"]) # Assume csv file is in root local directory
          tempX, tempY = get_consecutive_date_features(covid_df)
          if tempX.shape[0] > 0:
            dataX = np.vstack([dataX, tempX]) if dataX.size else tempX
            # dataY = np.concatenate((dataY, tempY), axis=0)

          if tempY.shape[0] > 0:
            dataY = np.vstack([dataY, tempY]) if dataY.size else tempY

          print(np.shape(dataX), np.shape(dataY))

    
with open("dataX.npy", "wb") as f:
    np.save(f, dataX)

Fetching data from ak.csv...
(280, 3, 1) (280, 3)
Fetching data from al.csv...
(560, 3, 1) (560, 3)
Fetching data from ar.csv...
(840, 3, 1) (840, 3)
Fetching data from az.csv...
(1120, 3, 1) (1120, 3)
Fetching data from ca.csv...
(1400, 3, 1) (1400, 3)
Fetching data from co.csv...
(1680, 3, 1) (1680, 3)
Fetching data from ct.csv...
(1960, 3, 1) (1960, 3)
Fetching data from dc.csv...
(2240, 3, 1) (2240, 3)
Fetching data from de.csv...
(2520, 3, 1) (2520, 3)
Fetching data from fl.csv...
(2800, 3, 1) (2800, 3)
Fetching data from ga.csv...
(3080, 3, 1) (3080, 3)
Fetching data from hi.csv...
(3360, 3, 1) (3360, 3)
Fetching data from ia.csv...
(3640, 3, 1) (3640, 3)
Fetching data from id.csv...
(3920, 3, 1) (3920, 3)
Fetching data from il.csv...
(4200, 3, 1) (4200, 3)
Fetching data from in.csv...
(4480, 3, 1) (4480, 3)
Fetching data from ks.csv...
(4760, 3, 1) (4760, 3)
Fetching data from ky.csv...
(5040, 3, 1) (5040, 3)
Fetching data from la.csv...
(5320, 3, 1) (5320, 3)
Fetching data from

In [ ]:
# Shuffle and split
train_test_split = int(0.7 * len(dataX))
shuffler = np.arange(dataX.shape[0])
np.random.shuffle(shuffler)

dataX_shuffled = dataX[shuffler]
dataY_shuffled = dataY[shuffler]

print(dataX_shuffled.shape, dataY_shuffled.shape)

tempX, tempY = dataX_shuffled[:train_test_split], dataY_shuffled[:train_test_split]
testX, testY = dataX_shuffled[train_test_split:], dataY_shuffled[train_test_split:]

print(tempX.shape, tempY.shape, testX.shape, testY.shape)

# Get validation data
train_val_split = int(0.6 * len(dataX))
trainX, trainY = tempX[:train_val_split], tempY[:train_val_split]
valX, valY = tempX[train_val_split:], tempY[train_val_split:]

print(trainX.shape, trainY.shape, valX.shape, valY.shape, testX.shape, testY.shape)

(14560, 3, 1) (14560, 3)
(10192, 3, 1) (10192, 3) (4368, 3, 1) (4368, 3)
(8736, 3, 1) (8736, 3) (1456, 3, 1) (1456, 3) (4368, 3, 1) (4368, 3)


In [ ]:
from keras.models import Sequential, load_model, save_model
from keras.layers import LSTM, TimeDistributed, Flatten, ConvLSTM2D
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
import keras
import tensorflow as tf
# import tensorflowjs as tfjs

batchsize = 128
dropout_amount = 0.2
n_length = 14
n_steps = 1
n_features = 1
n_channels = 1

# Based on Shastri et. al 2020
model = Sequential()
model.add(LSTM(100, return_sequences = True, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(LSTM(100, return_sequences = False, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='mse')

checkpoint_filepath = './tmp/checkpoints'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# fit network
history = model.fit(trainX, trainY, epochs=500, validation_data=(valX, valY), verbose=2, shuffle=False, callbacks=[model_checkpoint_callback])
# plot history
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')
pyplot.legend()
pyplot.show()

model.save('./tmp/covid-model-classify.h5')
# tfjs.converters.save_keras_model(model, checkpoint_filepath)

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

# make a prediction
y_predict = model.predict(testX)
# y_predict = y_predict.reshape((y_predict.shape[0], 1))
# testX = testX.reshape((testX.shape[0], n_days*9)) # 9 features
# # invert scaling for forecast
# inv_yhat = np.concatenate((yhat, testX[:, -8:]), axis=1)
inv_yhat = case_scaler.inverse_transform(y_predict)
# inv_yhat = inv_yhat[:,0]
# # invert scaling for actual
# testY = testY.reshape((len(testY), 1))
# inv_y = np.concatenate((testY, testX[:, -8:]), axis=1)
reshape_Y = testY.reshape(-1, 1)
inv_y = case_scaler.inverse_transform(reshape_Y)
# inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.6f' % rmse)

Test RMSE: 130.944342
